# Accessing Geoserver data


<!-- The THREDDS data storing netCDF file on PAVICS has some public and private directories. Data from public directories can be accessed anonymously, while data from private directories require authentication. This notebook shows how to access public and private data on the THREDDS server.

The PAVICS THREDDS server has a `testdata/` folder, in which we store test datasets to validate process requests. Within that directory is a `secure/` folder whose file access requires authentication. -->

In [1]:
# define some useful variables for following steps
import os
import requests

PAVICS_HOST = os.getenv("PAVICS_HOST", "pavics.ouranos.ca")

# TODO: remove temp url for testing
PAVICS_HOST = "172.17.0.1"

GEOSERVER_SECURED_URL = f"https://{PAVICS_HOST}/twitcher/ows/proxy/geoserver-secured"
GEOSERVER_ADMIN_URL = f"https://{PAVICS_HOST}/geoserver/rest"

assert PAVICS_HOST != "", "Invalid PAVICS HOST value."
print("GEOSERVER SECURED URL:", GEOSERVER_SECURED_URL)

MAGPIE_URL = f"https://{PAVICS_HOST}/magpie"

MAGPIE_ADMIN_USER = "admin"
MAGPIE_ADMIN_PASSWORD = "qwertyqwerty!"

GEOSERVER_ADMIN_USER = "admingeo"
GEOSERVER_ADMIN_PASSWORD = "geoserverpass"

MAGPIE_TEST_USER = "test"
MAGPIE_TEST_PASSWORD = "qwertyqwerty!"

MAGPIE_NO_PERM_USER = "no-perm-user"
MAGPIE_NO_PERM_PASSWORD = "no-perm-pass"

json_headers = {"Content-type": "application/json"}

GEOSERVER SECURED URL: https://172.17.0.1/twitcher/ows/proxy/geoserver-secured


In [2]:
def magpie_signin(user_name, password):
    signin_url = f"{MAGPIE_URL}/signin"
    data = {"user_name": user_name, "password": password}
    try:
        resp = requests.request(url=signin_url, method="POST", data=data, timeout=10, verify=VERIFY_SSL)
    except Exception as exc:
        raise RuntimeError(f"Failed to sign in to Magpie (url: `{url}`) with user `{data['user_name']}`. "
                           f"Exception : {exc}. ")
    if resp.status_code != 200:
        raise RuntimeError(f"Unexpected response while trying to sign in to Magpie : {resp.text}")
    return resp

In [3]:
VERIFY_SSL = False

geoserver_admin_session = requests.Session()
geoserver_admin_session.verify = VERIFY_SSL
geoserver_admin_session.auth = (GEOSERVER_ADMIN_USER, GEOSERVER_ADMIN_PASSWORD)

magpie_admin_session = requests.Session()
magpie_admin_session.verify = VERIFY_SSL
magpie_admin_session.cookies = magpie_signin(MAGPIE_ADMIN_USER, MAGPIE_ADMIN_PASSWORD).cookies

test_user_session = requests.Session()
test_user_session.verify = VERIFY_SSL

no_perm_user_session = requests.Session()
no_perm_user_session.verify = VERIFY_SSL

/home/cumminch/.conda/envs/pavics-sdi/lib/python3.11/site-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host '172.17.0.1'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


### Prepare Geoserver data

In [5]:
# create workspace
workspace_name = "test_workspace"
payload = {"workspace": {"name": workspace_name, "isolated": "True"}}
geoserver_admin_session.post(url=f"{GEOSERVER_ADMIN_URL}/workspaces", json=payload)

/home/cumminch/.conda/envs/pavics-sdi/lib/python3.11/site-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host '172.17.0.1'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


<Response [409]>

In [9]:
# create datastore
datastore_name = "test_datastore"
geoserver_datastore_path = "/geoserver-test-data"
payload = {
    "dataStore": {
        "name": datastore_name,
        "type": "Directory of spatial files (shapefiles)",
        "connectionParameters": {
            "entry": []
        }}}
geoserver_admin_session.post(url=f"{GEOSERVER_ADMIN_URL}/workspaces/{workspace_name}/datastores", json=payload).text

/home/cumminch/.conda/envs/pavics-sdi/lib/python3.11/site-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host '172.17.0.1'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


"Store 'test_datastore' already exists in workspace 'test_workspace'"

In [24]:
# configure datastore
payload = {
    "dataStore": {
        "name": datastore_name,
        "type": "Directory of spatial files (shapefiles)",
        "connectionParameters": {
            "entry": [
                {"$": "UTF-8",
                 "@key": "charset"},
                {"$": "shapefile",
                 "@key": "filetype"},
                {"$": "true",
                 "@key": "create spatial index"},
                {"$": "true",
                 "@key": "memory mapped buffer"},
                {"$": "GMT",
                 "@key": "timezone"},
                {"$": "true",
                 "@key": "enable spatial index"},
                {"$": f"http://{datastore_name}",
                 "@key": "namespace"},
                {"$": "true",
                 "@key": "cache and reuse memory maps"},
                {"$": f"file://{geoserver_datastore_path}",
                 "@key": "url"},
                {"$": "shape",
                 "@key": "fstype"},
            ]
        },
    }
}
geoserver_admin_session.put(url=f"{GEOSERVER_ADMIN_URL}/workspaces/{workspace_name}/datastores/{datastore_name}", json=payload)

/home/cumminch/.conda/envs/pavics-sdi/lib/python3.11/site-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host '172.17.0.1'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


<Response [200]>

In [12]:
# publish shapefile
filename = "Espace_Vert" ## TODO FILL ME : shapefile name with no extension
shapefile_payload = {
    "featureType": {
        "name": filename,
        "nativeCRS": """
                        GEOGCS[
                            "WGS 84", 
                            DATUM[
                                "World Geodetic System 1984",
                                SPHEROID["WGS 84", 6378137.0, 298.257223563, AUTHORITY["EPSG","7030"]],
                                AUTHORITY["EPSG","6326"]
                            ],
                            PRIMEM["Greenwich", 0.0, AUTHORITY["EPSG","8901"]],
                            UNIT["degree", 0.017453292519943295],
                            AXIS["Geodetic longitude", EAST],
                            AXIS["Geodetic latitude", NORTH],
                            AUTHORITY["EPSG","4326"]
                        ]
                    """,
        "srs": "EPSG:4326",
        "projectionPolicy": "REPROJECT_TO_DECLARED",
        "maxFeatures": 5000,
        "numDecimals": 6,
    }
}

resp = geoserver_admin_session.post(
    url=f"{GEOSERVER_ADMIN_URL}/workspaces/{workspace_name}/datastores/{datastore_name}/featuretypes",
    json=shapefile_payload,
    headers=json_headers
)
resp.__dict__

/home/cumminch/.conda/envs/pavics-sdi/lib/python3.11/site-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host '172.17.0.1'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


{'_content': b"Resource named 'Espace_Vert' already exists in store: 'test_datastore'",
 '_content_consumed': True,
 '_next': None,
 'status_code': 500,
 'headers': {'Server': 'nginx/1.23.4', 'Date': 'Fri, 08 Sep 2023 18:05:06 GMT', 'Content-Type': 'text/plain', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'X-Frame-Options': 'SAMEORIGIN', 'X-Content-Type-Options': 'nosniff', 'X-XSS-Protection': '1; mode=block', 'Access-Control-Allow-Origin': '*', 'Access-Control-Expose-Headers': 'Access-Control-Allow-Origin,Access-Control-Allow-Credentials'},
 'raw': <urllib3.response.HTTPResponse at 0x7f59b4319b40>,
 'url': 'https://172.17.0.1/geoserver/rest/workspaces/test_workspace/datastores/test_datastore/featuretypes',
 'encoding': 'ISO-8859-1',
 'history': [],
 'reason': '',
 'cookies': <RequestsCookieJar[]>,
 'elapsed': datetime.timedelta(microseconds=59019),
 'request': <PreparedRequest [POST]>,
 'connection': <requests.adapters.HTTPAdapter at 0x7f59b74293d0>}

In [26]:
# get feature types info
geoserver_admin_session.get(url=f"{GEOSERVER_ADMIN_URL}/workspaces/{workspace_name}/datastores/{datastore_name}/featuretypes",
                           headers=json_headers).content

/home/cumminch/.conda/envs/pavics-sdi/lib/python3.11/site-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host '172.17.0.1'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


b'<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN"\n    "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">\n<html xmlns="http://www.w3.org/1999/xhtml" xml:lang="en" lang="en">\n<head>\n    <title>GeoServer Configuration</title>\n    <meta name="ROBOTS" content="NOINDEX, NOFOLLOW"/>\n</head>\n<body>\n\nFeature Types:\n<ul>\n  <li><a href="https://172.17.0.1/geoserver/rest/workspaces/test_workspace/datastores/test_datastore/featuretypes/Espace_Vert.html">Espace_Vert</a></li>\n</ul>\n</body>\n</html>\n'

In [15]:
# get datastore info
geoserver_admin_session.get(url=f"{GEOSERVER_ADMIN_URL}/workspaces/{workspace_name}/datastores/{datastore_name}").json()

{'dataStore': {'name': 'test_datastore',
  'type': 'Directory of spatial files (shapefiles)',
  'enabled': True,
  'workspace': {'name': 'test_workspace',
   'href': 'http://localhost:8765/geoserver/rest/workspaces/test_workspace.json'},
  'connectionParameters': {'entry': [{'@key': 'charset', '$': 'UTF-8'},
    {'@key': 'filetype', '$': 'shapefile'},
    {'@key': 'create spatial index', '$': 'true'},
    {'@key': 'memory mapped buffer', '$': 'true'},
    {'@key': 'timezone', '$': 'GMT'},
    {'@key': 'enable spatial index', '$': 'true'},
    {'@key': 'namespace', '$': 'http://test_datastore'},
    {'@key': 'cache and reuse memory maps', '$': 'true'},
    {'@key': 'url',
     '$': 'file:///home/cumminch/PAVICS-e2e-workflow-tests/notebooks/test_workspace'},
    {'@key': 'fstype', '$': 'shape'}]},
  '_default': False,
  'dateCreated': '2023-09-08 17:31:25.508 UTC',
  'dateModified': '2023-09-08 17:31:29.663 UTC',
  'featureTypes': 'http://localhost:8765/geoserver/rest/workspaces/test_wor

In [17]:
# delete featuretype
geoserver_admin_session.delete(url=f"{GEOSERVER_ADMIN_URL}/layers/{filename}")
geoserver_admin_session.delete(url=f"{GEOSERVER_ADMIN_URL}/workspaces/{workspace_name}/datastores/{datastore_name}/featuretypes/{filename}")

/home/cumminch/.conda/envs/pavics-sdi/lib/python3.11/site-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host '172.17.0.1'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/home/cumminch/.conda/envs/pavics-sdi/lib/python3.11/site-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host '172.17.0.1'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


<Response [200]>

In [18]:
# delete datastore
geoserver_admin_session.delete(url=f"{GEOSERVER_ADMIN_URL}/workspaces/{workspace_name}/datastores/{datastore_name}")

/home/cumminch/.conda/envs/pavics-sdi/lib/python3.11/site-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host '172.17.0.1'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


<Response [200]>

In [15]:
# get on /workspaces
geoserver_admin_session.get(url=f"{GEOSERVER_ADMIN_URL}/workspaces").json()

/home/cumminch/.conda/envs/pavics-sdi/lib/python3.11/site-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host '172.17.0.1'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


{'workspaces': {'workspace': [{'name': 'test_workspace',
    'href': 'https://172.17.0.1/geoserver/rest/workspaces/test_workspace.json'}]}}

### Prepare Magpie users and permissions

In [16]:
magpie_admin_session.post(
    url=f"{MAGPIE_URL}/users",
    data={
        "user_name": MAGPIE_TEST_USER,
        "email": "test@user.com",
        "password": MAGPIE_TEST_PASSWORD,
        "group_name": "users"},
    allow_redirects=False)
test_user_session.cookies = magpie_signin(MAGPIE_TEST_USER, MAGPIE_TEST_PASSWORD).cookies

magpie_admin_session.post(
    url=f"{MAGPIE_URL}/users",
    data={
        "user_name": MAGPIE_NO_PERM_USER,
        "email": "no-perm-test@user.com",
        "password": MAGPIE_NO_PERM_PASSWORD,
        "group_name": "users"},
    allow_redirects=False)
no_perm_user_session.cookies = magpie_signin(MAGPIE_NO_PERM_USER, MAGPIE_NO_PERM_PASSWORD).cookies

/home/cumminch/.conda/envs/pavics-sdi/lib/python3.11/site-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host '172.17.0.1'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/home/cumminch/.conda/envs/pavics-sdi/lib/python3.11/site-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host '172.17.0.1'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/home/cumminch/.conda/envs/pavics-sdi/lib/python3.11/site-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host '172.17.0.1'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/

### Test user access

In [17]:
# requests.get(url=f"{BASE_URL}/geoserver-secured/wfs?version=2.0.0&request=GetFeature&typeNames=public:{LAYER_NAME}

test_user_session.get(url=f"https://172.17.0.1/twitcher/ows/proxy/geoserver-secured/wfs?version=2.0.0&request=GetCapabilities")


/home/cumminch/.conda/envs/pavics-sdi/lib/python3.11/site-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host '172.17.0.1'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


<Response [200]>